In [69]:
!pip install pandas sqlalchemy sqlacodegen

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [27]:
import pickle

In [28]:
import pandas as pd

In [29]:
data = [{"name":"Shaban"}]

with open("users.pkl","wb") as f:
    pickle.dump(data,f) #--->pickle.dump

In [30]:
data_frm_pkl = pickle.load(open('users.pkl','rb'))
print(data_frm_pkl,type(data_frm_pkl)) #--->pickle.load

[{'name': 'Shaban'}] <class 'list'>


In [31]:
df = pd.DataFrame(data)
df

,name
0,Shaban


In [32]:
df.to_pickle("df_pkl")

In [33]:
df2 = pd.read_pickle('df_pkl')
df2

,name
0,Shaban


In [34]:
df2.to_csv("my_df.csv",index=False)

## SQLAlchemy

SQLAlchemy & Django have a simila object-relational mappings (aka ORM) are ways to connect Python to a SQL database.

In [35]:
from dataclasses import dataclass

@dataclass
class Movie:
    name:str='Unknown'
    genre:str='Action'
    year:int=None

# class Movie:
#     name='Unknown'
#     genre='Action'
#     def __init__(self,name='',*args,**kwargs):
#         super().__init__(*args,**kwargs)
#         self.name = name

In [36]:
movie_obj = Movie(name='Interstellar',genre='Sci-Fi')

In [37]:
movie_obj.name

'Interstellar'

In [38]:
movie_obj.genre

'Sci-Fi'

In [39]:
data_ = [{
    'name' : 'Interstellar',
    'genre' : 'Sci-Fi' 
},
{
    'name' : 'Star Wars',
    'genre' : 'Sci-Fi'
},{
    'name' : 'Alita',
    'genre' : 'Sci-Fi'
}]

In [40]:
df3 = pd.DataFrame(data_)
df3

,name,genre
0,Interstellar,Sci-Fi
1,Star Wars,Sci-Fi
2,Alita,Sci-Fi


In [47]:
import sqlalchemy
from sqlalchemy import create_engine, Column, String, Integer
from sqlalchemy.orm import sessionmaker

In [42]:
engine = create_engine("sqlite:///app.db") # this can also be used for postgres,mysql engines ---> check the documentation

In [43]:
Session = sessionmaker(bind=engine)
my_sess = Session()

In [44]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [45]:
class Movie(Base): # table
    __tablename__= "movies"
   
    id = Column(Integer,primary_key=True) # auto created for us
    name = Column(String) # col
    genre = Column(String) # col
    description = Column(String)
    year = Column(Integer,nullable=True) # col
    
    def __repr__(self):
        return f"<Movie name={self.name}>"
        
# saved instance = row

In [46]:
#Add table to database
Base.metadata.create_all(engine)

In [26]:
movie_obj = Movie(name="Alita",genre="Sci-Fi")

In [22]:
print(movie_obj.name)

Alita


In [24]:
movie_obj.id

In [26]:
my_sess.add(movie_obj)# prepare to save
my_sess.commit() # save

In [27]:
movie_obj.id

1

In [29]:
movie_obj2 = Movie(name="John Wick",genre="Action")

In [31]:
my_sess.add(movie_obj2)# prepare to save
my_sess.commit() # save

In [33]:
movie_obj2.id

2

In [34]:
movie_obj3 = Movie(name="The Martian",genre="Action")
my_sess.add(movie_obj3)# prepare to save
my_sess.commit() 

## CRUD in SQLAlchemy

- ### Create
- ### Retrieve
- ### Update
- ### Delete

In [48]:
Session = sessionmaker(bind=engine)
session = Session()

In [53]:
# Create
movie = Movie(name='Guardians of the galaxy')
session.add(movie)
session.commit()

In [54]:
# Retrieve 
movie_a = session.query(Movie).get(2)
print(movie_a.id,movie_a.name)

2 Guardians of the galaxy


In [55]:
# List
qs = session.query(Movie).all()
print(qs)

[<Movie name=Guardians of the galaxy>, <Movie name=Guardians of the galaxy>]


In [57]:
# List & filter by column value
qs = session.query(Movie).filter_by(name='Guardians of the galaxy').all()
qs

[<Movie name=Guardians of the galaxy>, <Movie name=Guardians of the galaxy>]

In [61]:
# List & filter by column value containing something
my_query = input('What are you looking for?\n') or 'Unknown'
qs2 = session.query(Movie).filter(Movie.name.contains(my_query)).all()
qs2

What are you looking for?
Guardians 


[<Movie name=Guardians of the galaxy>, <Movie name=Guardians of the galaxy>]

In [64]:
# Retrieve & update
movie_a = session.query(Movie).get(2)
movie_a.description = "The galaxy super heroes"
session.commit()
print(movie_a.id,movie_a.name,movie_a.description)

2 Guardians of the galaxy The galaxy super heroes


In [65]:
qs2 = session.query(Movie).filter(Movie.name.contains(my_query)).all()
for movie_obj in qs2:
    movie_obj.name = "Guardians of the galaxy End game"
session.commit()

In [67]:
# Delete
movie_a = session.query(Movie).get(1)
session.delete(movie_a)
session.commit()

In [68]:
session.flush()

In [72]:
!sqlacodegen sqlite:///app2.db # sqlacodegen reverse engineers the code used to create the database

# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    year = Column(Integer)


In [73]:
engine = create_engine("sqlite:///app3.db")
Session = sessionmaker(bind=engine)
my_sess = Session()

In [77]:
# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class MovieApp3(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    year = Column(Integer)
    
    def __repr__(self):
        return f"<Movie name={self.name}>"
        

In [78]:
qs = session.query(MovieApp3).all()
qs

[<Movie name=Guardians of the galaxy End game>]

In [80]:
for old_obj in qs:
    movie_obj = Movie(name=old_obj.name,genre=old_obj.genre,year=old_obj.year)
    print(movie_obj.name)
    my_sess.add(movie_obj)
my_sess.commit()

Guardians of the galaxy End game


In [83]:
old_engine = create_engine("sqlite:///app2.db")
old_df = pd.read_sql_table("movies",old_engine)
old_df.head()

,id,name,genre,year
0,1,Alita,Sci-Fi,None
1,2,John Wick,Action,None
2,3,The Martian,Action,None


In [84]:
current_engine = create_engine("sqlite:///app.db")
current_df = pd.read_sql_table("movies",current_engine)
current_df.head()

,id,name,genre,description,year
0,2,Guardians of the galaxy End game,None,The galaxy super heroes,None


In [85]:
final_df = pd.concat([current_df,old_df])
final_df

,id,name,genre,description,year
0,2,Guardians of the galaxy End game,None,The galaxy super heroes,None
0,1,Alita,Sci-Fi,NaN,None
1,2,John Wick,Action,NaN,None
2,3,The Martian,Action,NaN,None


In [87]:
final_df = final_df[['name','genre','description','year']]
final_df.reset_index(inplace=True,drop=True)
final_df

,name,genre,description,year
0,Guardians of the galaxy End game,None,The galaxy super heroes,None
1,Alita,Sci-Fi,NaN,None
2,John Wick,Action,NaN,None
3,The Martian,Action,NaN,None


In [88]:
final_df.to_sql(
    "movies_2",
    current_engine,
    if_exists = "replace",
    dtype = {
        "name":String,
        "genre":String,
        "Description":String,
        "year":Integer
    }
)

In [89]:
!sqlacodegen sqlite:///app.db

# coding: utf-8
from sqlalchemy import BigInteger, Column, Integer, String, Table, Text
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer)


t_movies_2 = Table(
    'movies_2', metadata,
    Column('index', BigInteger, index=True),
    Column('name', String),
    Column('genre', String),
    Column('description', Text),
    Column('year', Integer)
)


In [90]:
import sqlite3 

conx = sqlite3.connect('app.db')

df = pd.read_sql_query("SELECT * FROM movies",conx)
df

,id,name,genre,description,year
0,2,Guardians of the galaxy End game,None,The galaxy super heroes,None
